# Data Storage

## Install libraries & packages

In [24]:
## Install libraries & install packages needed to run MongoDB
!pip3 install pymongo
import pymongo
import requests
import time
import matplotlib.pyplot as plt
import os
from IPython.display import clear_output
import pandas as pd
import seaborn as sns
os.environ['PATH'] += os.pathsep + '/usr/local/bin' 

## Start Docker container

In [25]:
!docker-compose up -d

 Container my_mongo  Running
 Container my_jupyter  Running
 Container bdeng-main-zookeeper-1  Running
 Container my_spark  Running
 Container bdeng-main-kafka-1  Running


### Check if the services are running

In [26]:
!docker ps

CONTAINER ID   IMAGE                              COMMAND                  CREATED        STATUS                  PORTS                                              NAMES
cab6dbd2207b   confluentinc/cp-kafka:latest       "/etc/confluent/dockâ€¦"   25 hours ago   Up 25 hours             0.0.0.0:9092->9092/tcp, 0.0.0.0:29093->29093/tcp   bdeng-main-kafka-1
487c3227b56c   jupyter/base-notebook:latest       "tini -g -- start-noâ€¦"   25 hours ago   Up 25 hours (healthy)   0.0.0.0:8888->8888/tcp                             my_jupyter
dc3e03349414   confluentinc/cp-zookeeper:latest   "/etc/confluent/dockâ€¦"   25 hours ago   Up 25 hours             2888/tcp, 3888/tcp, 0.0.0.0:32181->2181/tcp        bdeng-main-zookeeper-1
84faa88860f7   bitnami/spark:latest               "/opt/bitnami/scriptâ€¦"   25 hours ago   Up 25 hours             0.0.0.0:7077->7077/tcp, 0.0.0.0:8080->8080/tcp     my_spark
a8bcc88f2c24   mongo:latest                       "docker-entrypoint.sâ€¦"   25 hours ago   Up 25 h

## Connect to DB

# Process CSV Data

### Install necessary packages

In [27]:
!pip install pymongo

### Read data and display the first few rows

In [30]:
import pandas as pd

# Read the CSV file with the specified encoding
poi_df = pd.read_csv('data/top-locations-wien.csv', encoding='latin1', sep=';')


# Display the first few rows of the dataframe
print(poi_df.head())


             title    category  \
0        21er Haus      museum   
1     A.E. Köchert    shopping   
2             Aida       cafes   
3  Akademietheater  musicstage   
4        Albertina      museum   

                                        Beschreibung                  address  \
0  Das Museum wurde 2011 saniert und stellt unter...          Arsenalstraße 1   
1  Dieser Traditions-Juwelier schmückt heute mit ...          Neuer Markt 15    
2  Aida ist eine Wiener Konditoreikette. Das Desi...  Stock-im-Eisen-Platz 2    
3  Seit 1922 ist das Akademietheater die zweite S...            Lisztstraße 1   
4  Die Albertina besitzt nicht nur eine der größt...        Albertinaplatz 1    

    zip  city      geo_latitude     geo_longitude             tel_1  \
0  1030  Wien  48,1857710810320  16,3836224650817  +43 1 795 57-134   
1  1010  Wien        48,2065731        16,3705892               NaN   
2  1010  Wien        48,2080194        16,3720473  +43 1 512 79 25    
3  1030  Wien        48,

### Check for missing values

In [31]:
print(poi_df.isnull().sum())

title              0
category           0
Beschreibung       0
address            3
zip                0
city               0
geo_latitude       0
geo_longitude      0
tel_1             22
tel_1_comment    100
tel_2            129
tel_2_comment    130
tel_3            132
tel_3_comment    132
email             80
web_url            6
url                7
dtype: int64


### CSV-Data cleaning

In [32]:
# Fill or drop missing values
poi_df = poi_df.dropna()  # Dropping rows with missing values

# Check for duplicates
print(poi_df.duplicated().sum())

# Remove duplicates
poi_df = poi_df.drop_duplicates()

0


In [ ]:
# Remove unnecessary rows (geo_latitude, geo_longitude, tel_1, tel_1_comment, tel_2, tel_2_comment, tel_3, tel_3_comment, email, url)


### Show clean Dataframe

In [33]:
print(poi_df.head()

Empty DataFrame
Columns: [title, category, Beschreibung, address, zip, city, geo_latitude, geo_longitude, tel_1, tel_1_comment, tel_2, tel_2_comment, tel_3, tel_3_comment, email, web_url, url]
Index: []


## Safe data in MongoDB

In [ ]:
# Connect to MongoDB
client = MongoClient('localhost', 37017)
db = client['citybike_vienna']
collection = db['top_locations_wien']

# Insert data into MongoDB
collection.insert_many(poi_df.to_dict('records'))

print("Data inserted into MongoDB")

## Analyse and Vizualisation of CSV-Data

In [ ]:
# Anzahl der POIs pro Kategorie
category_counts = poi_df['Kategorie'].value_counts()

# Plotting the number of POIs per category
plt.figure(figsize=(10, 6))
sns.barplot(x=category_counts.index, y=category_counts.values, palette='viridis')
plt.title('Anzahl der POIs pro Kategorie')
plt.xlabel('Kategorie')
plt.ylabel('Anzahl der POIs')
plt.xticks(rotation=45)
plt.show()

## Linking the data with Citybike locations

In [ ]:
# Mock data for Citybike locations and usage

citybike_locations = {
    "Museum": 10,
    "Cafe": 15,
    "Nightlife": 5,
    "Park": 8,
    "Shopping": 12
}

# Assuming we have a similar structure for the POIs and Citybike locations
poi_df['Citybike_Standorte'] = poi_df['Kategorie'].map(citybike_locations)

# Plotting the number of Citybike locations per POI category
plt.figure(figsize=(10, 6))
sns.barplot(x=poi_df['Kategorie'].unique(), y=poi_df.groupby('Kategorie')['Citybike_Standorte'].sum(), palette='viridis')
plt.title('Anzahl der Citybike-Standorte pro POI Kategorie')
plt.xlabel('Kategorie')
plt.ylabel('Anzahl der Citybike-Standorte')
plt.xticks(rotation=45)
plt.show()

# Set up Spark

In [ ]:
import os
from pyspark.sql import SparkSession

# Create Spark session & context
spark = (SparkSession
         .builder
         .appName('nextbike-data-consumer')
         .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.0")
         .getOrCreate())

sc = spark.sparkContext

print("Spark session created successfully.")

## Transform and Analyze Data

In [ ]:
from pyspark.sql.functions import col, from_json, explode
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, IntegerType, ArrayType, LongType

# Define schema for the data
place_schema = StructType([
    StructField("uid", LongType(), True),
    StructField("name", StringType(), True),
    StructField("bike", IntegerType(), True),
    StructField("bike_numbers", ArrayType(StringType()), True),
    StructField("free_racks", IntegerType(), True),
    StructField("bike_racks", IntegerType(), True),
    StructField("lat", DoubleType(), True),
    StructField("lng", DoubleType(), True)
])

city_schema = StructType([
    StructField("name", StringType(), True),
    StructField("places", ArrayType(place_schema), True)
])

country_schema = StructType([
    StructField("name", StringType(), True),
    StructField("cities", ArrayType(city_schema), True)
])

data_schema = StructType([
    StructField("countries", ArrayType(country_schema), True)
])

# Create stream dataframe setting Kafka server, topic, and offset option
df = (spark
      .readStream
      .format("kafka")
      .option("kafka.bootstrap.servers", "localhost:29093")  # Kafka server
      .option("subscribe", "nextbike_data")  # Topic
      .option("startingOffsets", "earliest")  # Start from beginning
      .load())

# Convert binary to string key and value
df1 = (df
       .withColumn("key", df["key"].cast(StringType()))
       .withColumn("value", df["value"].cast(StringType())))

# Parse JSON data
df2 = df1.select(from_json(col("value"), data_schema).alias("data"))

# Explode the nested structure to flatten the DataFrame
df_countries = df2.select(explode(col("data.countries")).alias("country"))
df_cities = df_countries.select(col("country.name").alias("country_name"), explode(col("country.cities")).alias("city"))
df_places = df_cities.select(
    col("country_name"),
    col("city.name").alias("city_name"),
    explode(col("city.places")).alias("place")
)

# Select and rename required fields
df_formatted = df_places.select(
    col("country_name").alias("country"),
    col("city_name").alias("city"),
    col("place.uid").alias("place_uid"),
    col("place.name").alias("place_name"),
    col("place.bike").alias("bike"),
    col("place.bike_numbers").alias("bike_numbers"),
    col("place.free_racks").alias("free_racks"),
    col("place.bike_racks").alias("total_racks"),
    col("place.lat").alias("latitude"),
    col("place.lng").alias("longitude")
)

# Show the schema of the dataframe
df_formatted.printSchema()

# API

## Visualize the Streaming Data

In [ ]:
# Function to query the API and store data
def query_api_and_store_data():
    url = "https://api.nextbike.net/maps/nextbike-live.json"

    try:
        response = requests.get(url)
        data = response.json()
        
        # Extract relevant data
        countries = data['countries']
        records = []
        for country in countries:
            for city in country['cities']:
                for place in city['places']:
                    records.append({
                        'country': country['name'],
                        'city': city['name'],
                        'place_uid': place['uid'],
                        'place_name': place['name'],
                        'bike': place['bike'],
                        'bike_numbers': place.get('bike_numbers', []),
                        'free_racks': place.get('free_racks', None),
                        'total_racks': place.get('bike_racks', None),
                        'latitude': place['lat'],
                        'longitude': place['lng'],
                    })
        
        # Create DataFrame
        df = pd.DataFrame(records)
        
        # Display the head of the DataFrame
        print(df.head())
        
    except Exception as e:
        print("Error:", e)
query_api_and_store_data()